In [1]:
import polars as pl
import os
from pathlib import Path
import numpy as np

In [2]:
import Utils.constants
from Utils.constants import FOLDER
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT
from Utils.constants import MAX_COST_VALUE,MASS,K_DRAG, MAX_FORCE
from Utils.constants import index_to_speed,direction_to_lane_change
print(f"State space {N_LANES*N_SPEEDS*N_DIRECTIONS} N_SPEED_DEVIATION:{N_SPEED_DEVIATION} N_LANE_DEVIATION:{N_LANE_DEVIATION}")

State space 660 N_SPEED_DEVIATION:2 N_LANE_DEVIATION:1


In [3]:
#Frame wit the track information. To do: to move the creation of the lanes in here
path = Path(FOLDER + f"\\TrackLanes_{N_LANES}.parquet")
track_df = pl.read_parquet(path).limit(N_TIME_LIMIT)
N_TIME = len(track_df)
track_df

x_m,y_m,w_tr_right_m,w_tr_left_m,x_dir,y_dir,width,norm_dir,x_udir,y_udir,x_lane0,x_lane1,x_lane2,x_lane3,x_lane4,x_lane5,x_lane6,x_lane7,x_lane8,x_lane9,x_lane10,y_lane0,y_lane1,y_lane2,y_lane3,y_lane4,y_lane5,y_lane6,y_lane7,y_lane8,y_lane9,y_lane10
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3.439354,-0.495322,6.556,6.536,5.862596,8.10221,13.092,10.000792,0.586213,0.810157,8.742641,7.681983,6.621326,5.560669,4.500011,3.439354,2.378697,1.318039,0.257382,-0.803275,-1.863933,-4.332673,-3.565203,-2.797733,-2.030263,-1.262792,-0.495322,0.272148,1.039619,1.807089,2.574559,3.342029
6.370784,3.555763,6.558,6.537,5.862596,8.10221,13.095,10.000792,0.586213,0.810157,11.675286,10.614386,9.553485,8.492585,7.431684,6.370784,5.309884,4.248983,3.188083,2.127182,1.066282,-0.282468,0.485178,1.252825,2.020471,2.788117,3.555763,4.323409,5.091055,5.858701,6.626348,7.393994
9.30195,7.606888,6.56,6.538,5.862087,8.102292,13.098,10.00056,0.586176,0.810184,14.607844,13.546665,12.485486,11.424308,10.363129,9.30195,8.240771,7.179592,6.118414,5.057235,3.996056,3.768022,4.535795,5.303569,6.071342,6.839115,7.606888,8.374661,9.142434,9.910207,10.677981,11.445754
12.232871,11.658055,6.562,6.539,5.861618,8.10238,13.101,10.000356,0.586141,0.810209,17.540146,16.478691,15.417236,14.355781,13.294326,12.232871,11.171416,10.109961,9.048506,7.987051,6.925596,7.818539,8.586442,9.354345,10.122249,10.890152,11.658055,12.425958,13.193861,13.961765,14.729668,15.497571
15.163568,15.709268,6.564,6.54,5.861189,8.102473,13.104,10.00018,0.586108,0.810233,20.472213,19.410484,18.348755,17.287026,16.225297,15.163568,14.101839,13.04011,11.978381,10.916652,9.854923,11.869086,12.637123,13.405159,14.173195,14.941232,15.709268,16.477304,17.245341,18.013377,18.781413,19.54945
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-11.222438,-20.750233,6.535,6.559,5.866447,8.101822,13.094,10.002736,0.586484,0.809961,-5.919626,-6.980188,-8.040751,-9.101313,-10.161876,-11.222438,-12.283,-13.343563,-14.404125,-15.464688,-16.52525,-24.589945,-23.822003,-23.05406,-22.286118,-21.518175,-20.750233,-19.982291,-19.214348,-18.446406,-17.678463,-16.910521
-8.289395,-16.699311,6.54,6.553,5.865724,8.101872,13.093,10.002352,0.586434,0.809997,-2.986752,-4.04728,-5.107809,-6.168338,-7.228866,-8.289395,-9.349924,-10.410452,-11.470981,-12.53151,-13.592038,-20.538404,-19.770586,-19.002767,-18.234948,-17.46713,-16.699311,-15.931492,-15.163674,-14.395855,-13.628036,-12.860218
-5.356714,-12.648361,6.544,6.548,5.865019,8.10193,13.092,10.001986,0.586385,0.810032,-0.054244,-1.114738,-2.175232,-3.235726,-4.29622,-5.356714,-6.417208,-7.477702,-8.538196,-9.59869,-10.659184,-16.48684,-15.719144,-14.951449,-14.183753,-13.416057,-12.648361,-11.880665,-11.112969,-10.345273,-9.577578,-8.809882


In [4]:
verbose = False
def l0_range():
    return range(N_LANES)
def l1_range(index):
    return range(max(0,index-N_LANE_DEVIATION), min(N_LANES,index+N_LANE_DEVIATION+1))
distance_pairs = [(l0,l1) for l0 in l0_range() for l1 in l1_range(l0)]
if verbose and False:
    for (l0,l1) in distance_pairs:
        print(f"{l0} -> {l1}")
    if verbose:
        # print("Distance pairs")
        for (l0,l1) in distance_pairs:
            print(l0,l1)

def d_range(l0):
     return [index for index in range(N_DIRECTIONS) if (direction_to_lane_change(index) + l0 >= 0 and direction_to_lane_change(index) + l0 < N_LANES)]  
if verbose and False:
    print("Distance pairs")
    for l0 in range(N_LANES):
        for d0 in d_range(l0):
             print(l0,l0+direction_to_lane_change(d0))

def s0_range():
    return range(N_SPEEDS)
def s1_range(index):
    return range(max(0,index-N_SPEED_DEVIATION), min(N_SPEEDS,index+N_SPEED_DEVIATION+1))

#state*action space
action_range = [(l0,s0,d0,l0+direction_to_lane_change(d0),s1,d1) for l0 in l0_range() for s0 in s0_range() for d0 in d_range(l0) for s1 in s1_range(s0) for d1 in d_range(l0+direction_to_lane_change(d0)) ]
if verbose:
    print("state by action space", len(action_range))
    for (l0,s0,d0,l1,s1,d1) in action_range:
        print(f"{l0} {s0} {d0} -> {l1} {s1} {d1}")


In [5]:
#distance between lanes in subsequent time steps
def make_lane_distance_columns(l0,l1):
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1)
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1)
    return ((x_col1-x_col0).pow(2) + (y_col1-y_col0).pow(2)).sqrt().fill_null(strategy="forward").alias(f"d_l{l0}_l{l1}")
lane_distance_columns = [make_lane_distance_columns(l0,d0) for (l0,d0) in distance_pairs]

#distance between lanes in subsequent time steps
def make_velocity_change_column(l0,s0,d0,l1,s1,d1):
    l2 = l1+direction_to_lane_change(d1)
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    x_col2 = pl.col("x_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    y_col2 = pl.col("y_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    d12_col = pl.col(f"d_l{l1}_l{l2}")
    return ((speed0*(x_col2-x_col1)/d12_col - speed1*(x_col1-x_col0)/d01_col).pow(2) + 
            (speed0*(y_col2-y_col1)/d12_col - speed1*(y_col1-y_col0)/d01_col).pow(2)).sqrt().fill_null(pl.lit(0)).alias(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
velocity_change_columns = [make_velocity_change_column(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]
#make name functions for these
velocity_change_columns_names = [f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}" for (l0,s0,d0,l1,s1,d1) in action_range]

#average speed
def make_average_speed(s0,s1):
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    return (0.5*(pl.lit(speed0) + pl.lit(speed1))).alias(f"as_s{s0}_s{s1}")
average_speeds_columns = [make_average_speed(s0,s1) for s0 in s0_range() for s1 in s1_range(s0)]

#time cost
def make_time_costs(l0,l1,s0,s1):
    av_speed = pl.col(f"as_s{s0}_s{s1}")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    return pl.when(d01_col == 0).then(MAX_COST_VALUE).otherwise(av_speed/d01_col).alias(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
time_costs_columns = [make_time_costs(l0,l1,s0,s1) for l0 in l0_range() for l1 in l1_range(l0) for s0 in s0_range() for s1 in s1_range(s0)]

def make_acceleration(l0,s0,d0,l1,s1,d1):
    velocity_change = pl.col(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    time_cost = pl.col(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
    return (velocity_change/time_cost).alias(f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
acceleration_columns = [make_acceleration(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]

opt_ldf = track_df.lazy().with_columns(lane_distance_columns).with_columns(velocity_change_columns).with_columns(average_speeds_columns).with_columns(time_costs_columns).with_columns(acceleration_columns)
opt_df  = opt_ldf .collect()
opt_df 

x_m,y_m,w_tr_right_m,w_tr_left_m,x_dir,y_dir,width,norm_dir,x_udir,y_udir,x_lane0,x_lane1,x_lane2,x_lane3,x_lane4,x_lane5,x_lane6,x_lane7,x_lane8,x_lane9,x_lane10,y_lane0,y_lane1,y_lane2,y_lane3,y_lane4,y_lane5,y_lane6,y_lane7,y_lane8,y_lane9,y_lane10,d_l0_l0,d_l0_l1,d_l1_l0,d_l1_l1,d_l1_l2,…,acc_l10s17d1_l9s19d1,acc_l10s17d1_l9s19d2,acc_l10s18d0_l10s16d0,acc_l10s18d0_l10s16d1,acc_l10s18d0_l10s17d0,acc_l10s18d0_l10s17d1,acc_l10s18d0_l10s18d0,acc_l10s18d0_l10s18d1,acc_l10s18d0_l10s19d0,acc_l10s18d0_l10s19d1,acc_l10s18d1_l9s16d0,acc_l10s18d1_l9s16d1,acc_l10s18d1_l9s16d2,acc_l10s18d1_l9s17d0,acc_l10s18d1_l9s17d1,acc_l10s18d1_l9s17d2,acc_l10s18d1_l9s18d0,acc_l10s18d1_l9s18d1,acc_l10s18d1_l9s18d2,acc_l10s18d1_l9s19d0,acc_l10s18d1_l9s19d1,acc_l10s18d1_l9s19d2,acc_l10s19d0_l10s17d0,acc_l10s19d0_l10s17d1,acc_l10s19d0_l10s18d0,acc_l10s19d0_l10s18d1,acc_l10s19d0_l10s19d0,acc_l10s19d0_l10s19d1,acc_l10s19d1_l9s17d0,acc_l10s19d1_l9s17d1,acc_l10s19d1_l9s17d2,acc_l10s19d1_l9s18d0,acc_l10s19d1_l9s18d1,acc_l10s19d1_l9s18d2,acc_l10s19d1_l9s19d0,acc_l10s19d1_l9s19d1,acc_l10s19d1_l9s19d2
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3.439354,-0.495322,6.556,6.536,5.862596,8.10221,13.092,10.000792,0.586213,0.810157,8.742641,7.681983,6.621326,5.560669,4.500011,3.439354,2.378697,1.318039,0.257382,-0.803275,-1.863933,-4.332673,-3.565203,-2.797733,-2.030263,-1.262792,-0.495322,0.272148,1.039619,1.807089,2.574559,3.342029,5.000457,5.168668,5.169412,5.000457,5.168744,…,0.574246,2.677346,0.587495,1.404091,0.285141,1.308385,0.000486,1.277435,0.270561,1.305297,1.451123,0.607434,2.684185,1.35219,0.294902,2.634569,1.320201,0.00051,2.619079,1.349014,0.27951,2.633001,0.554851,1.390965,0.269714,1.305162,0.000486,1.277435,1.437553,0.573691,2.677229,1.348858,0.278955,2.632943,1.320201,0.00051,2.619079
6.370784,3.555763,6.558,6.537,5.862596,8.10221,13.095,10.000792,0.586213,0.810157,11.675286,10.614386,9.553485,8.492585,7.431684,6.370784,5.309884,4.248983,3.188083,2.127182,1.066282,-0.282468,0.485178,1.252825,2.020471,2.788117,3.555763,4.323409,5.091055,5.858701,6.626348,7.393994,5.000636,5.168919,5.169601,5.000576,5.168936,…,0.573963,2.677841,0.587797,1.404432,0.28546,1.308691,0.000241,1.277681,0.270214,1.305471,1.451528,0.60767,2.684793,1.35257,0.29515,2.635163,1.320531,0.000359,2.619649,1.349277,0.279239,2.633539,0.555155,1.391303,0.270033,1.305465,0.000241,1.277681,1.437958,0.573929,2.677836,1.349236,0.279204,2.633536,1.320531,0.000359,2.619649
9.30195,7.606888,6.56,6.538,5.862087,8.102292,13.098,10.00056,0.586176,0.810184,14.607844,13.546665,12.485486,11.424308,10.363129,9.30195,8.240771,7.179592,6.118414,5.057235,3.996056,3.768022,4.535795,5.303569,6.071342,6.839115,7.606888,8.374661,9.142434,9.910207,10.677981,11.445754,5.000508,5.168872,5.169555,5.000451,5.168893,…,0.573947,2.678409,0.587804,1.404698,0.285471,1.308976,0.000221,1.277972,0.270194,1.305751,1.451802,0.607685,2.685365,1.352861,0.295165,2.635746,1.320826,0.000371,2.620234,1.349561,0.279223,2.634119,0.555162,1.391571,0.270044,1.305751,0.000221,1.277972,1.438234,0.573943,2.678409,1.349528,0.279219,2.634119,1.320826,0.000371,2.620234
12.232871,11.658055,6.562,6.539,5.861618,8.10238,13.101,10.000356,0.586141,0.810209,17.540146,16.478691,15.417236,14.355781,13.294326,12.232871,11.171416,10.109961,9.048506,7.987051,6.925596,7.818539,8.586442,9.354345,10.122249,10.890152,11.658055,12.425958,13.193861,13.961765,14.729668,15.497571,5.000395,5.16884,5.169523,5.000342,5.168865,…,0.573937,2.678977,0.587806,1.404963,0.285478,1.309262,0.000204,1.278263,0.270179,1.306033,1.452074,0.607697,2.685937,1.35315,0.295177,2.636328,1.321119,0.000385,2.62082,1.349846,0.279212,2.634699,0.555165,1.391839,0.270052,1.306038,0.000204,1.278263,1.438508,0.573955,2.678983,1.349818

In [6]:
path = Path(FOLDER + f"\\opt_{N_LANES}.parquet")
#adf1.drop(velocity_change_columns_names).write_parquet(path)
opt_df.write_parquet(path)

In [10]:
#Tools to navigate the cost array
def make_cost_array(): return np.full((N_LANES*N_SPEEDS*N_DIRECTIONS,N_LANES*N_SPEEDS*N_DIRECTIONS,N_TIME), MAX_COST_VALUE)
#costs_arr = make_cost_array()

tc_arr = make_cost_array()
acc_arr = make_cost_array()
as_arr = make_cost_array()

def state_cost_tuple_to_index(l0,s0,d0):
    index = l0*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS + d0 
    return index

for (l0,s0,d0,l1,s1,d1) in action_range:
    index0 = state_cost_tuple_to_index(l0,s0,d0) 
    index1 = state_cost_tuple_to_index(l1,s1,d1)
    tc_arr[index0][index1] = opt_df[f"tc_l{l0}_{l1}_s{s0}_s{s1}"].to_numpy()
    acc_arr[index0][index1] = opt_df[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"].to_numpy()
    as_arr[index0][index1] = opt_df[f"as_s{s0}_s{s1}"].to_numpy()

In [ ]:
np.save(Path(FOLDER + f"\\tc_{N_LANES}"), tc_arr)
np.save(Path(FOLDER + f"\\as_{N_LANES}"), as_arr)
np.save(Path(FOLDER + f"\\acc_{N_LANES}"), acc_arr)